In [ ]:
from ancient_greek_playground import download_canonical_greek_lit

download_canonical_greek_lit()

In [ ]:
from ancient_greek_playground import DataRepo

repo = DataRepo()

In [ ]:
repo.author_names()

In [ ]:
repo.works_by_author()

In [ ]:
plato_works = repo.read_author_works('plato')

In [ ]:
len(plato_works)

In [ ]:
import spacy
from ancient_greek_playground import replace_grave_with_acutus
from ancient_greek_playground.paths import TMP_DIR_PROCESSED
import json
from collections import defaultdict
from tqdm import tqdm

nlp = spacy.load("el_core_news_sm")

freqs = defaultdict(lambda: defaultdict(int))

for work_name, work_text in tqdm(plato_works.items()):
    f = freqs[work_name]
    doc = nlp(work_text)
    for token in doc:
        if token.is_alpha:
            f[replace_grave_with_acutus(token.lemma_)] += 1

for work_name, f in freqs.items():
    freqs[work_name] = dict(sorted(f.items(), key=lambda x: x[1], reverse=True))

with (TMP_DIR_PROCESSED / "freqs.json").open('w') as f:
    json.dump(freqs, f, ensure_ascii=False, indent=4)


In [ ]:
import math

def get_tf_term(term: str, work_name: str):
    return math.log10(freqs[work_name][term] + 1)

def get_df_term(term: str):
    df = 0
    for work_freqs in freqs.values():
        if term in work_freqs:
            df += 1
    return math.log10(len(freqs) / df)

def get_tf_idf(term: str, work_name: str):
    return freqs[work_name][term] * get_df_term(term)

In [ ]:
rev_freqs = {
    work_name: dict(sorted(
        {
            term: get_tf_idf(term, work_name)
            for term in freqs[work_name].keys()
        }.items(),
        key=lambda x: x[1],
        reverse=True,
    ))
    for work_name in freqs
}
with (TMP_DIR_PROCESSED / "rev_freqs.json").open('w') as f:
    json.dump(rev_freqs, f, ensure_ascii=False, indent=4)

In [2]:
import spacy
from spacy import displacy

nlp = spacy.load("el_core_news_sm")
sentence = 'ἐγὼ δʼ οὖν καὶ αὐτὸς ὑπʼ αὐτῶν ὀλίγου ἐμαυτοῦ ἐπελαθόμην, οὕτω πιθανῶς ἔλεγον.'
doc = nlp(sentence)
# TODO: this is not good, read how to work with spacy and check out
# https://github.com/perseids-tools/morpheus
# https://github.com/PerseusDL/morpheus
displacy.render(doc, style="dep", jupyter=True)